In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from sklearn.neighbors import KDTree
import math
import kdtree
import time
import operator
import pyautogui as pg
from sklearn.cluster import KMeans
from collections import defaultdict

pg.PAUSE = 0.000000001

In [2]:
################################################ KD TREE #####################################################
#not used in this project

class create:
    def __init__(self, indices):
        self.k = 2
        self.points = []
        self.points = list(indices)
        self.tree = self.build(self.points)

    def distance(self, point1, point2):
        x1, y1 = point1
        x2, y2 = point2
        dx = abs(x1 - x2)
        dy = abs(y1 - y2)
        return math.sqrt(dx * dx + dy * dy)

    def build(self, points, depth = 0):
        if len(points) <= 0:
            return None
        n = np.shape(points)[0]
        axis = depth % self.k
        sorted_points = sorted(points, key = lambda point: point[axis])
        pos = math.floor(n / 2)
        return {
            'point' : sorted_points[pos],
            'left' : self.build(sorted_points[:pos], depth + 1),
            'right' : self.build(sorted_points[pos + 1:], depth + 1) 
        }

    def closer_distance(self, pivot, p1, p2):
        if p1 is None:
            return p2
        if p2 is None:
            return p1
        d1 = self.distance(pivot, p1)
        d2 = self.distance(pivot, p2)
        if d1 < d2:
            return p1
        else:
            return p2

    def search_nn(self, point):
        return self.kdtree_closest_point(self.tree, point)

    def kdtree_closest_point(self, root, point, depth = 0):
        if root is None:
            return None
        axis = depth % self.k
        next_branch = None
        opposite_branch = None
        if point[axis] < root['point'][axis]:
            next_branch = root['left']
            opposite_branch = root['right']
        else:
            next_branch = root['right']
            opposite_branch = root['left']
        best = self.closer_distance(point, self.kdtree_closest_point(next_branch, point, depth + 1), root['point'])
        if self.distance(point, best) > abs(point[axis] - root['point'][axis]):
            best = self.closer_distance(point, self.kdtree_closest_point(opposite_branch, point, depth + 1), best)
        return best

   # def remove(self, point):
   #     kd_delete_point(self.tree, point)
   #     
   # def kd_delete_point(self, root, point):
   #     if root.point == point:




####################################################################################################

In [3]:
class AutoDraw(object):
    def __init__(self, name, blur = 0):
        # Tunable parameters
        self.detail = 1
        self.scale = 7/12 #
        self.sketch_before = False
        self.with_color = True
        self.num_colors = 10
        self.outline_again = False

        # Load Image. Switch axes to match computer screen
        self.img = self.load_img(name)
        self.blur = blur #
        self.img = np.swapaxes(self.img, 0, 1) #
        self.img_shape = self.img.shape
        self.dim = pg.size() #
        # Scale to draw inside part of screen
        self.startX = ((1 - self.scale) / 2)*math.floor(1 * self.dim[0]) #
        self.startY = ((1 - self.scale) / 2)*math.floor(1 * self.dim[1]) #
        self.dim = (self.dim[0] * self.scale, self.dim[1] * self.scale) #

        # fit the picture into this section of the screen
        if self.img_shape[1] > self.img_shape[0]:
            # if it's taller that it is wide, truncate the wide section
            self.dim = (int(self.dim[1] * (self.img_shape[0] / self.img_shape[1])), self.dim[1])
        else:
            # if it's wider than it is tall, truncate the tall section
            self.dim = (self.dim[0], int(self.dim[0] *(self.img_shape[1] / self.img_shape[0])))

        # Get dimension to translate picture. Dimension 1 and 0 are switched due to comp dimensions
        ratio = self.img.shape[0] / self.img.shape[1]
        pseudo_x = int(self.img.shape[1] * self.detail)
        self.pseudoDim = (pseudo_x, int(pseudo_x * ratio))
        
          # Initialize directions for momentum when creating path
        self.maps = {0: (1, 1), 1: (1, 0), 2: (1, -1), 3: (0, -1), 4: (0, 1), 5: (-1, -1), 6: (-1, 0), 7: (-1, 1)} #
        self.momentum = 1
        self.curr_delta = self.maps[self.momentum] 

        # Create Outline
        self.drawing = self.process_img(self.img)
        
    def show(self):
        cv2.namedWindow("show", 0)
        cv2.imshow("show", self.img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def load_img(self, path):
        img = cv2.imread(path)
        return img
    
    def rescale(self, img, dim):
        resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        return resized

    def translate(self, coord):
        ratio = (coord[0] / self.pseudoDim[1], coord[1] / self.pseudoDim[0]) # this is correct
        deltas = (int(ratio[0] * self.dim[0]), int(ratio[1] * self.dim[1]))
        return self.startX + deltas[0], self.startY + deltas[1]
    
    def process_img(self, img):                  
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if self.blur == 2:
            gray = cv2.GaussianBlur(gray, (9, 9), 0)
            canny = cv2.Canny(gray, 25, 45)
        elif self.blur == 1:
            gray = cv2.GaussianBlur(gray, (3, 3), 0)
            canny = cv2.Canny(gray, 25, 45)
        else:  # no blur
            canny = cv2.Canny(gray, 50, 75)
        canny = self.rescale(canny, self.pseudoDim)
        r, res = cv2.threshold(canny, 50, 255, cv2.THRESH_BINARY_INV)
        #cv2.namedWindow("show", 0)
        #cv2.imshow("show", res)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        return res
    
    def execute(self, commands):
        press = 0  # flag indicating whether we are putting pressure on paper
        for (i, comm) in enumerate(commands):
            if type(comm) == str:
                if comm == 'UP':
                    press = 0
                if comm == 'DOWN':
                    press = 1
            else:
                if press == 0:
                    pg.moveTo(comm[0], comm[1], 0)
                else:
                    pg.dragTo(comm[0], comm[1], 0)
        return

    def drawOutline(self):
        indices = np.argwhere(self.drawing < 127).tolist()  # get the black colors
        index_tuples = map(tuple, indices)

        self.hashSet = set(index_tuples)
        self.KDTree = kdtree.create(list(indices), dimensions=2)
        self.commands = []
        #self.curr_pos = list(indices)[0]
        self.curr_pos = (0, 0)
        point = self.translate(self.curr_pos)
        self.commands.append(point)

        print('Please change pen to thin and color to black.')
        input("Press enter once ready")
        print('')

        # DRAW THE BLACK OUTLINE
        self.createPath()
        input("Ready! Press Enter to draw")
        print('5 seconds until drawing beings')
        time.sleep(5)

        self.execute(self.commands)

    def createPath(self):
        # check for closest point. Go there. Add click down. Change curr. Remove from set and tree. Then, begin
        new_pos = tuple(self.KDTree.search_nn(self.curr_pos)[0].data)
        #self.commands.append(new_pos)
        #self.commands.append("DOWN")
        self.curr_pos = new_pos
        #self.KDTree = self.KDTree.remove(list(new_pos))
        #self.hashSet.remove(new_pos)

        while len(self.hashSet) > 0:
            prev_direction = self.momentum
            candidate = self.checkMomentum(self.curr_pos)
            if self.isValid(candidate):
                new = tuple(map(operator.add, self.curr_pos, candidate))
                new_pos = self.translate(new)
                if prev_direction == self.momentum and type(self.commands[-1]) != str:  # the direction didn't change
                    self.commands.pop()
                self.commands.append(new_pos)
            else:
                self.commands.append("UP")
                new = tuple(self.KDTree.search_nn(self.curr_pos)[0].data)
                new_pos = self.translate(new)
                self.commands.append(new_pos)
                self.commands.append("DOWN")
            self.curr_pos = new
            self.KDTree = self.KDTree.remove(list(new))
            self.hashSet.remove(new)
            if len(self.hashSet) % 1000 == 0 :
                print('Making path...number points left: ', len(self.hashSet))
        return

    def isValid(self, delta):
        return len(delta) == 2

    def checkMomentum(self, point):
        # Returns best next relative move w.r.t. momentum and if in hashset
        self.curr_delta = self.maps[self.momentum]
        moments = self.maps.values()
        deltas = [d for d in moments if (tuple(map(operator.add, point, d)) in self.hashSet)]
        deltas.sort(key=self.checkDirection, reverse=True)
        if len(deltas) > 0:
            best = deltas[0]
            self.momentum = [item[0] for item in self.maps.items() if item[1] == best][0]
            return best
        return [-1]

    def checkDirection(self, element):
        return self.dot(self.curr_delta, element)

    def dot(self, pt1, pt2):
        pt1 = self.unit(pt1)
        pt2 = self.unit(pt2)
        return pt1[0] * pt2[0] + pt1[1] * pt2[1]

    def unit(self, point):
        norm = (point[0] ** 2 + point[1] ** 2)
        norm = np.sqrt(norm)
        return point[0] / norm, point[1] / norm
        
    def add_color(self):
        color = self.rescale(self.img, self.pseudoDim)
        collapsed = np.sum(color, axis=2)/3
        fill = np.argwhere(collapsed < 230)  # color 2-d indices
        fill = np.swapaxes(fill, 0, 1)  # swap to index into color
        RGB = color[fill[0], fill[1], :]
        k_means = KMeans(n_clusters=self.num_colors).fit(RGB)
        colors = k_means.cluster_centers_
        labels = k_means.labels_
        fill = np.swapaxes(fill, 0, 1).tolist()  # swap back to make dictionary
        label_2_index = defaultdict(list)

        for i, j in zip(labels, fill):
            label_2_index[i].append(j)
            
        for (i, color) in enumerate(colors):
            print('Please change the pen to thick and color to BGR (not RGB) values: ', color)
            im = np.zeros(shape=(128, 128, 3))
            im[:, :, 0] = color[2]
            im[:, :, 1] = color[1]
            im[:, :, 2] = color[0]
            im = im.astype(np.uint8)
            f = plt.figure()
            f.set_figwidth(16)
            f.set_figheight(3)
            plt.imshow(im)
            plt.show()
            input("Press enter once ready")
            print('')

            points = label_2_index[i]
            index_tuples = map(tuple, points)
            self.hashSet = set(index_tuples)
            self.KDTree = kdtree.create(points, dimensions=2)
            self.commands = []
            self.curr_pos = (0, 0)
            point = self.translate(self.curr_pos)
            self.commands.append(point)
            self.commands.append("UP")
            self.createPath()

            input('Ready! Press enter to draw: ')
            print('5 seconds until drawing begins...')
            time.sleep(5)

            self.execute(self.commands)
  #  if self.outline_again:
  #      self.drawOutline()

In [ ]:
path = "messi.jpg"

#reducing the colors in the input image

img = cv2.imread(path)
img2 = img.reshape((-1, 3))
img2 = np.float32(img2)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

#no. of clusters
k = 10

attempts = 10

ret, label, center = cv2.kmeans(img2, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
center = np.uint8(center)

res = center[label.flatten()]
res2 = res.reshape((img.shape))
img = res2

cv2.imwrite("mod" + path, img)

img = Image.open("mod" + path)
img.show()

img = cv2.imread("mod" + path)

print("press enter to continue...")
dummy = input()


pic = AutoDraw(path, blur = 2)
pic.drawOutline()
pic.add_color()